In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\yfsta\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


  Stored in directory: C:\Users\yfsta\AppData\Local\Temp\pip-ephem-wheel-cache-fnize5q6\wheels\54\7c\d8\f86364af8fbba7258e14adae115f18dd2c91552406edc3fdaa
Successfully built en-core-web-sm

    Linking successful
    C:\Users\yfsta\Anaconda3\lib\site-packages\en_core_web_sm -->
    C:\Users\yfsta\Anaconda3\lib\site-packages\spacy\data\en

    You can now load the model via spacy.load('en')



Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,possibility,consolation,brother,clear,hasten,extraordinary,less,daughter,anybody,contempt,time,latitude,acceptable,everything,steadiness,utility,suit,handsomer,public,perceive,gift,friendship,observe,impose,dinah,peace,wit,opportunity,beauty,extent,mislead,coarse,remember,earnestly,issue,importance,estimate,disgust,secret,unite,...,therefore,dealing,grey,distress,lamp,jealous,rear,consideration,dare,livery,first,pool,hearthrug,provide,poison,down,description,john,prize,deserve,bewitch,november,hall,ancestry,fact,insult,tradespeople,ii,deep,solicitude,chance,knife,arm,breeding,sarcastically,indies,wild,hole,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.992

Test set score: 0.8383233532934131


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(250, 1647) (250,)
Training set score: 0.976

Test set score: 0.8862275449101796


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.968

Test set score: 0.8143712574850299


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6330275229357798


col_0,Austen,Carroll
row_0,,
Austen,120,20
Carroll,60,18


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

### Support Vector Machines

In [19]:
from sklearn.svm import SVC

# Instantiate our model and fit the data.
svm = SVC(kernel = 'linear')
svm.fit(X_train, y_train)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

Training set score: 0.984

Test set score: 0.8562874251497006


Linear SVM does not reach the 88.6% accuracy reached by logistic regression.

In [20]:
svm = SVC(kernel = 'rbf')
svm.fit(X_train, y_train)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Training set score: 0.688

Test set score: 0.6826347305389222


RBF Kernel SVM performs much worse than the other models.

### K-Nearest Neighbors

In [25]:
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
knn_fit = GridSearchCV(knn,{'weights':['uniform','distance'],'n_neighbors':range(1,10)},cv=5,n_jobs=5).fit(X_train, y_train)

print('Training set score:', knn_fit.best_estimator_.score(X_train, y_train))
print('\nTest set score:', knn_fit.best_estimator_.score(X_test, y_test))

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Training set score: 0.984

Test set score: 0.6646706586826348


K-Nearest Neighbors also performs poorly.

### Additional spaCy features

In [89]:
# Utility function to create a list of the 2000 most common words.
def new_bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def new_bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
        
    df['punct_cnt'] = 0
    df['noun_cnt'] = 0
    df['verb_cnt'] = 0
    df['adj_cnt'] = 0
    df['adv_cnt'] = 0
    df['intj_cnt'] = 0
    df['pronoun_cnt'] = 0
    df['conj_cnt'] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Count parts of speech relative to sentence length
        for token in sentence:
            if token.pos_ == 'NOUN':
                df.loc[i, 'noun_cnt'] += 1 / len(sentence)
            elif token.pos_ == 'VERB':
                df.loc[i, 'verb_cnt'] += 1 / len(sentence)
            elif token.pos_ == 'ADJ':
                df.loc[i, 'adj_cnt'] += 1 / len(sentence)
            elif token.pos_ == 'ADV':
                df.loc[i, 'adv_cnt'] += 1 / len(sentence)
            elif token.pos_ == 'INTJ':
                df.loc[i, 'intj_cnt'] += 1 / len(sentence)
            elif token.pos_ == 'PRON':
                df.loc[i, 'pronoun_cnt'] += 1 / len(sentence)
            elif token.pos_ == 'CONJ':
                df.loc[i, 'conj_cnt'] += 1 / len(sentence)
            elif token.is_punct:
                df.loc[i, 'punct_cnt'] += 1 / len(sentence)
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [90]:
# Set up the bags.
new_alicewords = new_bag_of_words(alice_doc)
new_persuasionwords = new_bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
new_common_words = set(new_alicewords + new_persuasionwords)

# Create our data frame with features. This can take a while to run.
new_word_counts = new_bow_features(sentences, new_common_words)
new_word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,possibility,consolation,brother,clear,hasten,extraordinary,less,daughter,anybody,contempt,time,latitude,acceptable,everything,steadiness,utility,suit,handsomer,public,perceive,gift,friendship,observe,impose,dinah,peace,wit,opportunity,beauty,extent,mislead,coarse,remember,earnestly,issue,importance,estimate,disgust,secret,unite,...,dare,livery,first,pool,hearthrug,provide,poison,down,description,john,prize,deserve,bewitch,november,hall,ancestry,fact,insult,tradespeople,ii,deep,solicitude,chance,knife,arm,breeding,sarcastically,indies,wild,hole,text_sentence,text_source,punct_cnt,noun_cnt,verb_cnt,adj_cnt,adv_cnt,intj_cnt,pronoun_cnt,conj_cnt
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,0.149254,0.179104,0.194030,0.044776,0.044776,0.000000,0.044776,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,0.111111,0.126984,0.174603,0.111111,0.111111,0.000000,0.063492,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,0.121212,0.060606,0.151515,0.030303,0.181818,0.060606,0.060606,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,0.333333,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll,0.333333,0.000000,0.333333,0.166667,0.000000,0.000000,0.166667,0


In [91]:
Y2 = new_word_counts['text_source']
X2 = np.array(new_word_counts.drop(['text_sentence','text_source'], 1))

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, Y2, test_size=0.4, random_state=0)

In [92]:
# Run logistic regression
lr2 = LogisticRegression(penalty='l2')
train2 = lr2.fit(X2_train, y2_train)

print('Training set score:', lr2.score(X2_train, y2_train))
print('\nTest set score:', lr2.score(X2_test, y2_test))

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training set score: 0.992

Test set score: 0.9161676646706587


By calculating frequency of each part of speech within each sentence, accuracy can be increased to 91.6%.

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [93]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


### Load Herman Melville's Moby Dick

In [94]:
# Load and clean the data.
moby_dick = gutenberg.raw('melville-moby_dick.txt')
moby_dick = re.sub(r'CHAPTER .*', '', moby_dick)
moby_dick = text_cleaner(moby_dick[:int(len(moby_dick)/10)])

In [100]:
# Parse the cleaned novels. This can take a bit.
moby_dick_doc = nlp(moby_dick)

In [101]:
moby_dick_sents = [[sent, "Melville"] for sent in alice_doc.sents]
new_moby_dickwords = new_bag_of_words(moby_dick_doc)

### Compare to Alice in Wonderland

In [104]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + moby_dick_sents)

# Combine bags to create a set of unique words.
new_common_words = set(new_alicewords + new_moby_dickwords)

# Create our data frame with features. This can take a while to run.
new_word_counts = new_bow_features(sentences, new_common_words)
new_word_counts.head()

Y2 = new_word_counts['text_source']
X2 = np.array(new_word_counts.drop(['text_sentence','text_source'], 1))

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, Y2, test_size=0.4, random_state=0)

# Run logistic regression
lr2 = LogisticRegression(penalty='l2')
train2 = lr2.fit(X2_train, y2_train)

print('Training set score:', lr2.score(X2_train, y2_train))
print('\nTest set score:', lr2.score(X2_test, y2_test))

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250


C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training set score: 0.6815286624203821

Test set score: 0.22857142857142856


The model using Alice in Wonderland does not work well with an accuracy of less than 23%.

### Compare to Persuasion

In [105]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(persuasion_sents + moby_dick_sents)

# Combine bags to create a set of unique words.
new_common_words = set(new_persuasionwords + new_moby_dickwords)

# Create our data frame with features. This can take a while to run.
new_word_counts = new_bow_features(sentences, new_common_words)
new_word_counts.head()

Y2 = new_word_counts['text_source']
X2 = np.array(new_word_counts.drop(['text_sentence','text_source'], 1))

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, Y2, test_size=0.4, random_state=0)

# Run logistic regression
lr2 = LogisticRegression(penalty='l2')
train2 = lr2.fit(X2_train, y2_train)

print('Training set score:', lr2.score(X2_train, y2_train))
print('\nTest set score:', lr2.score(X2_test, y2_test))

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training set score: 0.992

Test set score: 0.8982035928143712


The model using Persuasion has almost 90% accuracy.

In [107]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(persuasion_sents + emma_sents + moby_dick_sents)

# Combine bags to create a set of unique words.
new_emmawords = new_bag_of_words(emma_doc)
new_common_words = set(new_persuasionwords + new_emmawords + new_moby_dickwords)

# Create our data frame with features. This can take a while to run.
new_word_counts = new_bow_features(sentences, new_common_words)
new_word_counts.head()

Y2 = new_word_counts['text_source']
X2 = np.array(new_word_counts.drop(['text_sentence','text_source'], 1))

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, Y2, test_size=0.4, random_state=0)

# Run logistic regression
lr2 = LogisticRegression(penalty='l2')
train2 = lr2.fit(X2_train, y2_train)

print('Training set score:', lr2.score(X2_train, y2_train))
print('\nTest set score:', lr2.score(X2_test, y2_test))

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550


C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training set score: 0.9550898203592815

Test set score: 0.8923766816143498


The model using both Persuasion and Emma has over 89% accuracy.